In [70]:
import pandas as pd
df_final_pre3 = pd.read_csv('df_final_pre3.csv')
df_final_pre3

,year_month_day,T_d,P_t,month,community,techiness,entropy,count_q,count_a,ln_q,ln_a,ln_entropy,year_month
0,2021-09-01,0,0,9,0,0.525040,7.298398,1213.0,1029.0,7.100852,6.936343,1.987655,2021-09
1,2021-09-01,0,0,9,1,0.412791,7.399966,401.0,225.0,5.993961,5.416100,2.001475,2021-09
2,2021-09-01,0,0,9,3,0.490224,7.231138,394.0,250.0,5.976351,5.521461,1.978396,2021-09
3,2021-09-01,0,0,9,4,0.439161,7.709924,368.0,305.0,5.908083,5.720312,2.042508,2021-09
4,2021-09-01,0,0,9,5,0.461350,7.043218,378.0,258.0,5.934894,5.552960,1.952065,2021-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10707,2023-08-31,1,1,8,11,0.476285,4.175869,18.0,12.0,2.890372,2.484907,1.429322,2023-08
10708,2023-08-31,1,1,8,13,0.421145,5.528088,29.0,18.0,3.367296,2.890372,1.709842,2023-08
10709,2023-08-31,1,1,8,14,0.403302,3.344698,8.0,1.0,2.079442,0.000000,1.207377,2023-08
10710,2023-08-31,1,1,8,15,0.307775,2.947703,5.0,4.0,1.609438,1.386294,1.081026,2023-08


### Community Size Calculation
- Number of unique users aggregated monthly

In [4]:
# Load Data (preChatGPT)
import pickle
with open(file = 'ques_df_pre.pickle', mode = 'rb') as file:
    ques_df = pickle.load(file)

In [5]:
ques_df

,id,creation_date,body,tags,community,full_body,code_body
0,69006423,2021-09-01 00:00:35.237,<p>Using interceptors for the first time to re...,"[nodeDOTjs, reactjs, express, axios, refreshto...",0,Using interceptors for the first time to refre...,
1,69006426,2021-09-01 00:00:55.583,"<p>The APP below uses <code>navbarPage</code>,...","[r, shiny]",17,"The APP below uses navbarPage, however I will ...",dashboardPage\nnavbarPage\nlibrary(shiny)\nlib...
2,69006431,2021-09-01 00:01:23.670,<p>I am developing a java project and every th...,"[java, installation, compilation, project, ope...",3,I am developing a java project and every thing...,
3,69006437,2021-09-01 00:03:18.750,"<p>As you know, all firebase hosting is provid...","[firebase, nuxtDOTjs, firebasehosting]",5,"As you know, all firebase hosting is provided ...","[\n '@nuxtjs/firebase',\n {\n ..."
4,69006439,2021-09-01 00:03:36.930,<p>I am trying to split the string as below</p...,"[r, dataDOTtable]",17,I am trying to split the string as below\nx <-...,"x <- data.table(VAROLD=c('DBA','ADB'))\nx[, pa..."
...,...,...,...,...,...,...,...
2713807,77019848,2023-08-31 23:54:30.057,<p>I need help solving a CORS issue.</p>\n<p>I...,"[docker, dockercompose, owaspdependencycheck, ...",7,I need help solving a CORS issue.\nI am trying...,- API_BASE_URL=http://XXX.XXX.XX.XX:8081\nAcce...
2713808,77019849,2023-08-31 23:55:21.660,<p>I am currently attempting to search a Share...,"[microsoftgraphapi, sharepointonline]",1,I am currently attempting to search a SharePoi...,"{\n ""requests"": [\n {\n ..."
2713809,77019852,2023-08-31 23:56:42.653,"<p>Similar to <a href=""https://stackoverflow.c...","[python, numpy, floatingpoint]",17,Similar to Numpy astype rounding to wrong valu...,In [2]: import numpy as np\n\n...\n\nIn [49]: ...
2713810,77019854,2023-08-31 23:57:28.633,<p>I utilized the guidelines presented in <a h...,"[angular, typescript, progressivewebapps, ngsw...",0,I utilized the guidelines presented in this ar...,import {BrowserModule} from '@angular/platform...


In [13]:
# Import Modules
import pandas as pd
import numpy as np
import sqlite3
import pandas as pd
from nltk import FreqDist
import pickle
import math
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm

# Import Dataset
conn = sqlite3.connect('/data1/StackOverflow/stackexchange-to-sqlite/stack.db')
query = '''
SELECT id, owner_user_id
FROM questions
WHERE creation_date > '2021-09-01'
AND creation_date < '2023-09-01';
'''
df_users = pd.read_sql_query(query, conn)
conn.close()

In [20]:
df = pd.merge(ques_df, df_users, on = 'id', how = 'left')
df

,id,creation_date,body,tags,community,full_body,code_body,owner_user_id
0,69006423,2021-09-01 00:00:35.237,<p>Using interceptors for the first time to re...,"[nodeDOTjs, reactjs, express, axios, refreshto...",0,Using interceptors for the first time to refre...,,14087917.0
1,69006426,2021-09-01 00:00:55.583,"<p>The APP below uses <code>navbarPage</code>,...","[r, shiny]",17,"The APP below uses navbarPage, however I will ...",dashboardPage\nnavbarPage\nlibrary(shiny)\nlib...,13091928.0
2,69006431,2021-09-01 00:01:23.670,<p>I am developing a java project and every th...,"[java, installation, compilation, project, ope...",3,I am developing a java project and every thing...,,8110267.0
3,69006437,2021-09-01 00:03:18.750,"<p>As you know, all firebase hosting is provid...","[firebase, nuxtDOTjs, firebasehosting]",5,"As you know, all firebase hosting is provided ...","[\n '@nuxtjs/firebase',\n {\n ...",2745485.0
4,69006439,2021-09-01 00:03:36.930,<p>I am trying to split the string as below</p...,"[r, dataDOTtable]",17,I am trying to split the string as below\nx <-...,"x <- data.table(VAROLD=c('DBA','ADB'))\nx[, pa...",12139975.0
...,...,...,...,...,...,...,...,...
2713807,77019848,2023-08-31 23:54:30.057,<p>I need help solving a CORS issue.</p>\n<p>I...,"[docker, dockercompose, owaspdependencycheck, ...",7,I need help solving a CORS issue.\nI am trying...,- API_BASE_URL=http://XXX.XXX.XX.XX:8081\nAcce...,1006272.0
2713808,77019849,2023-08-31 23:55:21.660,<p>I am currently attempting to search a Share...,"[microsoftgraphapi, sharepointonline]",1,I am currently attempting to search a SharePoi...,"{\n ""requests"": [\n {\n ...",15216800.0
2713809,77019852,2023-08-31 23:56:42.653,"<p>Similar to <a href=""https://stackoverflow.c...","[python, numpy, floatingpoint]",17,Similar to Numpy astype rounding to wrong valu...,In [2]: import numpy as np\n\n...\n\nIn [49]: ...,433202.0
2713810,77019854,2023-08-31 23:57:28.633,<p>I utilized the guidelines presented in <a h...,"[angular, typescript, progressivewebapps, ngsw...",0,I utilized the guidelines presented in this ar...,import {BrowserModule} from '@angular/platform...,2532775.0


### 1) NA values

In [21]:
df['owner_user_id'].isna().sum()

32284

In [37]:
df.owner_user_id.isna().groupby(df.community).sum().reset_index()

,community,owner_user_id
0,0,9940
1,1,2465
2,2,0
3,3,2334
4,4,2781
5,5,2290
6,6,2621
7,7,667
8,8,453
9,9,214


In [39]:
df.groupby('community').size().reset_index()

,community,0
0,0,665468
1,1,233828
2,2,21
3,3,211591
4,4,223931
5,5,225552
6,6,235708
7,7,131413
8,8,60820
9,9,18556


In [41]:
df.isna().sum()

id                   0
creation_date        0
body                 0
tags                 0
community            0
full_body            0
code_body            0
owner_user_id    32284
dtype: int64

In [44]:
df = df.dropna()

In [53]:
df['owner_user_id'] = df['owner_user_id'].astype(int).astype('str')

/tmp/ipykernel_1186583/2661467321.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['owner_user_id'] = df['owner_user_id'].astype(int).astype('str')


### 2) Calculate Community Size (Monthly Number of Users)

In [48]:
df['year_month'] = pd.to_datetime(df['creation_date']).dt.to_period('M') # Monthly
df['year_month'] = df['year_month'].astype(str)

/tmp/ipykernel_1186583/258911141.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['year_month'] = pd.to_datetime(df['creation_date']).dt.to_period('M') # Monthly
/tmp/ipykernel_1186583/258911141.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['year_month'] = df['year_month'].astype(str)


In [69]:
agg_monthly = df.groupby(['year_month', 'community'])['owner_user_id'].nunique().reset_index()
agg_monthly

,year_month,community,owner_user_id
0,2021-09,0,22317
1,2021-09,1,7820
2,2021-09,2,1
3,2021-09,3,7714
4,2021-09,4,7720
...,...,...,...
394,2023-08,11,343
395,2023-08,13,630
396,2023-08,14,278
397,2023-08,15,122


In [65]:
# same code as above (to confirm)
len(df[(df['year_month'] == '2021-09')&(df['community'] == 0)]['owner_user_id'].unique())

22317

In [67]:
# For the whole community (not monthly aggregated)
df.groupby('community')['owner_user_id'].nunique().reset_index()

,community,owner_user_id
0,0,332009
1,1,126980
2,2,12
3,3,126386
4,4,135899
5,5,112119
6,6,148525
7,7,86878
8,8,40714
9,9,10533


In [82]:
# Merge
df_final_pre4 = pd.merge(df_final_pre3, agg_monthly, on = ['year_month', 'community'], how = 'left')
df_final_pre4 = df_final_pre4.rename(columns = {'owner_user_id': 'numUser'})
df_final_pre4['ln_numUser'] = np.log(df_final_pre4['numUser'])
df_final_pre4

,year_month_day,T_d,P_t,month,community,techiness,entropy,count_q,count_a,ln_q,ln_a,ln_entropy,year_month,numUser,ln_numUser
0,2021-09-01,0,0,9,0,0.525040,7.298398,1213.0,1029.0,7.100852,6.936343,1.987655,2021-09,22317,10.013104
1,2021-09-01,0,0,9,1,0.412791,7.399966,401.0,225.0,5.993961,5.416100,2.001475,2021-09,7820,8.964440
2,2021-09-01,0,0,9,3,0.490224,7.231138,394.0,250.0,5.976351,5.521461,1.978396,2021-09,7714,8.950792
3,2021-09-01,0,0,9,4,0.439161,7.709924,368.0,305.0,5.908083,5.720312,2.042508,2021-09,7720,8.951570
4,2021-09-01,0,0,9,5,0.461350,7.043218,378.0,258.0,5.934894,5.552960,1.952065,2021-09,7509,8.923858
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10707,2023-08-31,1,1,8,11,0.476285,4.175869,18.0,12.0,2.890372,2.484907,1.429322,2023-08,343,5.837730
10708,2023-08-31,1,1,8,13,0.421145,5.528088,29.0,18.0,3.367296,2.890372,1.709842,2023-08,630,6.445720
10709,2023-08-31,1,1,8,14,0.403302,3.344698,8.0,1.0,2.079442,0.000000,1.207377,2023-08,278,5.627621
10710,2023-08-31,1,1,8,15,0.307775,2.947703,5.0,4.0,1.609438,1.386294,1.081026,2023-08,122,4.804021


In [83]:
# Save Data
df_final_pre4.to_csv('df_final_pre4.csv', index=False)